# Facility Location
## Interger Programming Example

In [1]:
from gurobipy import *
import numpy as np

### Setting up indices, parameters.
Set up indices for sites and markets. In this example, we have 5 production plants, and 4 markets.

In [2]:
n = 5 # 5 sites for potential plants
m = 4 # 4 markets
sites = range(n) # Equivalently, sites = [0,1,2,3,4]
markets = range(m) # Equivalently, markets = [0,1,2,3]

Import parameters values. Note that the dimensions of the parameter vectors/matrices should be consistent with n and m.  

In [3]:
# Warehouse demand in thousands of units
demand = np.array([15, 18, 14, 20])

# Plant capacity in thousands of units
capacity = np.array([20, 22, 17, 19, 18])

# Fixed costs for each plant
fixedCosts = np.array([12000, 15000, 17000, 13000, 16000])

# Production cost (per 1000 units) at each plan
prodCosts = np.array([220, 350, 170, 330, 240])

# Transportation costs per thousand units
transportationCosts = np.array([[4000, 2500, 1200, 2200],
              [2000, 2600, 1800, 2600],
              [3000, 3400, 2600, 3100],
              [2500, 3000, 4100, 3700],
              [4500, 4000, 3000, 3200]])

### Setting up the model.

In [4]:
m = Model("facility")

Academic license - for non-commercial use only - expires 2022-08-29
Using license file /Users/phoebe/gurobi.lic


Set up decision variables.

To define binary variables, add "vtype=GRB.BINARY" in the variable definition. This way Gurobi knows that each x[i] should be constrained to take value 0 or 1. 

In [5]:
# Plant open decision variables, taking value 0 or 1
# x[i] == 1 if plant i is open.
x = m.addVars(sites, vtype=GRB.BINARY)

Define other variables.
Note that we are setting the variable non-negativity constraints here directly, by setting the lower bound to be 0.0. This way, each element of y is set to be >= 0, and each element of z is set to be >= 0.0. This is equivalent to explicitly setting a non-negativity constraint "for i in sites: m.addConstr(y[i] >= 0.0)"

In [6]:
# production variable y
y = m.addVars(sites, lb=0.0)

# shipment variable z
z = m.addVars(sites, markets, lb=0.0)

Set objective function.
Here because the objective function is very long, we initialize an LinExpr object to hold it first, and pass the LinExpr obj to setObjective function later.

In [7]:
# The objective is to minimize the total fixed and variable costs
objFn = LinExpr()
objFn += sum(fixedCosts[i] * x[i] for i in sites)
objFn += sum(prodCosts[i] * y[i] for i in sites)
objFn += sum(sum(transportationCosts[i,j] * z[i,j] for i in sites) for j in markets)
m.setObjective(objFn, GRB.MINIMIZE)

Now set all the constraints.

In [8]:
# Production constraints
for i in sites:
    m.addConstr(y[i] <= capacity[i] * x[i])

# Outbound transportation constraints
for i in sites:
    m.addConstr(sum(z[i,j] for j in markets) <= y[i])

# Demand satisfication constraints
for j in markets:
    m.addConstr(sum(z[i,j] for i in sites) >= demand[j])

### Solving the model.
 
Integer program models can be much harder to solve than linear programs. It may very well be that a seemingly simple model can take hours or days to solve. So it is always a good computational practice to explicit set the solver run time.

In [9]:
m.Params.TimeLimit = 60 # seconds

Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [10]:
# Solve
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 14 rows, 30 columns and 55 nonzeros
Model fingerprint: 0x075158d1
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)

Root relaxation: objective 2.192900e+05, 16 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 219290.000    0    1          - 219290.000      -     -    0s
H    0     0                    229710.00000 219290.000  4.54%     -    0s
     0     0     cutoff    0      229710.00

In this case, it only takes a fraction of a second to solve, so our time limit was not activated.

Print the solution, i.e., objective value and the optimal variable values:

In [12]:
# Print optimal cost
print("Objective value, total cost = ", m.objVal)

# Print optimal solution
print("Facility opening decisions: plant number, decision")
for i in sites:
    print("Plant ", i, " opening decision: '", x[i].x)
    
print("Production decisions: plant number, production quantity")
for i in sites:
    print("Plant ", i, " produces ", y[i].x)
    
print("Shipment decisions: plant number, market number, shipment quantity")
for i in sites:
    for j in markets:
        print("Plant ", i, " ships to market ", j, ": ", z[i,j].x)

Objective value, total cost =  229710.0
Facility opening decisions: plant number, decision
Plant  0  opening decision: ' 1.0
Plant  1  opening decision: ' 1.0
Plant  2  opening decision: ' 1.0
Plant  3  opening decision: ' 1.0
Plant  4  opening decision: ' -0.0
Production decisions: plant number, production quantity
Plant  0  produces  20.0
Plant  1  produces  22.0
Plant  2  produces  14.0
Plant  3  produces  11.0
Plant  4  produces  0.0
Shipment decisions: plant number, market number, shipment quantity
Plant  0  ships to market  0 :  0.0
Plant  0  ships to market  1 :  0.0
Plant  0  ships to market  2 :  14.0
Plant  0  ships to market  3 :  6.0
Plant  1  ships to market  0 :  15.0
Plant  1  ships to market  1 :  7.0
Plant  1  ships to market  2 :  0.0
Plant  1  ships to market  3 :  0.0
Plant  2  ships to market  0 :  0.0
Plant  2  ships to market  1 :  0.0
Plant  2  ships to market  2 :  0.0
Plant  2  ships to market  3 :  14.0
Plant  3  ships to market  0 :  0.0
Plant  3  ships to m